In [ ]:
import time
import json
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

def synonym(customlist, searchterm):
    customlist = []
    resp = requests.get('https://api.datamuse.com/words?ml=' + searchterm)
    if resp.status_code != 200:
        # This means something went wrong.
        raise ApiError('GET /' + searchterm + '/ {}'.format(resp.status_code))
    for item in resp.json():
        customlist.append('{}'.format(item['word']))
        customlist.append('{}'.format(item['word']).title())
    return customlist

def setup_method():
    options = webdriver.ChromeOptions()
    options.add_argument('no-sandbox')
    options.add_argument('headless')
    global driver
    global vars
    driver = webdriver.Chrome(options=options, executable_path=ChromeDriverManager().install())

def login_zuckwatch(password):
    driver.get("https://zuckwatch.com/")
    driver.find_element(By.CSS_SELECTOR, ".input").click()
    driver.find_element(By.CSS_SELECTOR, ".input").send_keys(password)
    driver.find_element(By.LINK_TEXT, "Login").click()
    WebDriverWait(driver, 2000).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, ".nuxt-progress-failed")))
    isPresent = len(driver.find_elements(By.CSS_SELECTOR, ".nuxt-progress-failed")) > 0
    if isPresent:
        print("Incorrect password: " + password)
    else:
        print("The password is: " + password)
        return 'Correct'

def teardown_method():
    driver.quit()


In [ ]:
new_list = []
deduped_list = []
search_list = [
        'PASSWORD GOES HERE'
        ]

for term in search_list:
    new_list.append(term)
    new_list.append(term.title())
    for item in synonym(new_list, term):
        new_list.append(item)

deduped_list = list(dict.fromkeys(new_list))

print('searching ' + str(len(deduped_list)) + ' files...')

In [ ]:
setup_method()


In [ ]:
for password in new_list:
    if login_zuckwatch(password) == 'Correct':
        print('The password was found! It is ' + password)
        break


In [ ]:
teardown_method()
